# ETL I

## Task I - Define correct schema for json data

* load json dataset
* look at the infered schema (is it inferred correctly or is it wrong?)
* define the schema explicitly
* see what happens if the schema is defined wrong

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc, count, explode, split, regexp_replace, collect_list

from pyspark.sql.types import StructType, StructField, StringType, LongType, TimestampType

import os

In [ ]:
spark = (
    SparkSession
    .builder
    .appName('ETL I')
    .getOrCreate()
)

In [ ]:
print(spark.version)

The input dataset is in the json format and is in the `data/questios-json` folder. Below is the path definition:

In [ ]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

data_input_path = os.path.join(project_path, 'data/questions-json')

output_path = os.path.join(project_path, 'output/questions-transformed')

#### First let Spark infer the schema:

In [ ]:
questionsDF = (
    spark
    .read
    .format('json')
    .option('path', data_input_path)
    .load()
)

Note: Here we have only 8 json files. In case where you have lots of json files and you know that each file has the same schema, consider loading only one file to check the schema. Inferring the schema from many files can be expensive.

In [ ]:
questionsDF.printSchema()

#### Define the schema:

As you can see the `creation_date` was inferred as `StringType` however in reality it is a Timestamp. Define the schema by hand and provide it to create the DataFrame

In [ ]:
json_schema = StructType(
    [
        StructField('question_id', LongType(), True),
        StructField('creation_date', TimestampType(), True),
        StructField('title', StringType(), True),
        StructField('body', StringType(), True),
        StructField('tags', StringType(), True),
        StructField('accepted_answer_id', LongType(), True),
        StructField('answers', LongType(), True),
        StructField('comments', LongType(), True),
        StructField('user_id', LongType(), True),
        StructField('views', LongType(), True),
    ]
)

In [ ]:
questionsDF = (
    spark
    .read
    .schema(json_schema)
    .format('json')
    .option('path', data_input_path)    
    .load()
)

In [ ]:
questionsDF.show(truncate=10, n=5)

In [ ]:
questionsDF.count()

#### What happens if we make a mistake:

* set `title` as `LongType` in the defined schema

Hint
* Different things will happen depending on the `mode` option, where `mode` is one of the following:
    * FAILFAST
    * DROPMALFORMED
    * PERMISSIVE (default)


In [ ]:
# Define the schema with a mistake in the title column:

wrong_schema = StructType(
    [
        StructField('question_id', LongType(), True),
        StructField('creation_date', TimestampType(), True),
        StructField('title', LongType(), True),
        StructField('body', StringType(), True),
        StructField('tags', StringType(), True),
        StructField('accepted_answer_id', LongType(), True),
        StructField('answers', LongType(), True),
        StructField('comments', LongType(), True),
        StructField('user_id', LongType(), True),
        StructField('views', LongType(), True),
    ]
)

In [ ]:
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'PERMISSIVE') # this is the default
    .option('path', data_input_path)    
    .load()
).show(truncate=15, n=5)

In [ ]:
# shows no records
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'DROPMALFORMED')
    .option('path', data_input_path)    
    .load()
).show(truncate=15, n=5)

In [ ]:
# throws an error
(
    spark
    .read
    .schema(wrong_schema)
    .format('json')
    .option('mode', 'FAILFAST')
    .option('path', data_input_path)    
    .load()
).show(truncate=15, n=5)

#### Note

* To read more about schema inferrence and schema evolution of json files in Spark SQL, read my article: https://medium.com/swlh/notes-about-json-schema-handling-in-spark-sql-be1e7f13839d


## Task II - Transfrom json to parquet and convert String column to an array

* convert column tags to array of tags 

#### Convert tags to an array

Hint
* use split to get an array
* explode the array
* use regexp_replace
* groupBy + collect_list
* join with original questions DataFrame

In [ ]:
resultDF = (
    questionsDF
    .withColumn('tags_arr', split('tags', '><'))
    .withColumn('tag', explode('tags_arr'))
    .withColumn('tag', regexp_replace('tag', '(<|>)', ''))
    .groupBy('question_id')
    .agg(collect_list('tag').alias('tags'))
    .join(questionsDF.drop('tags'), 'question_id')
)

#### Save the data

Hint:

* repartition the data to 8 partitions before saving
 * this will create 8 files

In [ ]:
(
    resultDF
    .repartition(8)
    .write
    .mode('overwrite')
    .option('path', output_path)
    .save()
)

<b>Check if we saved the data correctly:</b>

In [ ]:
checkDF = (
    spark
    .read
    .parquet(output_path)
)

In [ ]:
checkDF.count()

In [ ]:
checkDF.show(truncate=10, n=5)

In [ ]:
checkDF.select('tags').show(truncate=False, n=10)

In [ ]:
spark.stop()